SQL Murder Mystery There's been a Murder in SQL City! The SQL Murder Mystery is designed to be both a self-directed lesson to learn SQL concepts and commands and a fun game for experienced SQL users to solve an intriguing crime. A crime has taken place and the detective needs your help. The detective gave you the crime scene report, but you somehow lost it. You vaguely remember that the crime was a ​murder​ that occurred sometime on ​Jan.15, 2018,​ and that it took place in ​SQL City​.

Analyze the results of your SQL queries to piece together the sequence of events and identify potential suspects. Conclude the detective challenge by summarising your findings and presenting a compelling case.

In [6]:
import sqlite3
import pandas as pd
import numpy as np
con = sqlite3.connect('/content/sample_data/murder-mystery.db')

# 1.  Retrieve Crime Scene Report:


In [7]:
filtered_data = pd.read_sql_query("""
SELECT *
FROM crime_scene_report""", con)

In [8]:
filtered_data.head()


,date,type,description,city
0,20180115,robbery,A Man Dressed as Spider-Man Is on a Robbery Spree,NYC
1,20180115,murder,Life? Dont talk to me about life.,Albany
2,20180115,murder,"Mama, I killed a man, put a gun against his he...",Reno
3,20180215,murder,REDACTED REDACTED REDACTED,SQL City
4,20180215,murder,Someone killed the guard! He took an arrow to ...,SQL City


In [9]:
#Run a query to retrieve the crime scene report for the murder that occurred on Jan.15, 2018, in SQL City.
#Gather all available details from the report.

filtered_data = pd.read_sql_query("""
SELECT *
FROM crime_scene_report
WHERE type = 'murder'
  AND date = '20180115'
  AND city = 'SQL City';

""", con)

In [10]:
print(filtered_data.shape)
filtered_data.head()

(1, 4)


,date,type,description,city
0,20180115,murder,Security footage shows that there were 2 witne...,SQL City


# 2. Witness Personal Details:
Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".

In [11]:
#Check the personal details of witnesses involved in the case.
#Retrieve their names, addresses, and any other relevant information.

filtered_data = pd.read_sql_query("""
SELECT * FROM person
""", con)

In [12]:
print(filtered_data.shape)
filtered_data.head()

(10011, 6)


,id,name,license_id,address_number,address_street_name,ssn
0,10000,Christoper Peteuil,993845,624,Bankhall Ave,747714076
1,10007,Kourtney Calderwood,861794,2791,Gustavus Blvd,477972044
2,10010,Muoi Cary,385336,741,Northwestern Dr,828638512
3,10016,Era Moselle,431897,1987,Wood Glade St,614621061
4,10025,Trena Hornby,550890,276,Daws Hill Way,223877684


In [13]:
#  Retrieve details of the first witness
filtered_data = pd.read_sql_query("""
SELECT *
FROM person
WHERE address_street_name LIKE '%Northwestern Dr%'
ORDER BY address_number DESC
LIMIT 1;
""", con)
print(filtered_data)


      id            name  license_id  address_number address_street_name  \
0  14887  Morty Schapiro      118009            4919     Northwestern Dr   

         ssn  
0  111564949  


In [14]:
# Retrieve details of the second witness named Annabel
filtered_data = pd.read_sql_query("""SELECT *
FROM person
WHERE name LIKE '%Annabel%' AND
address_street_name LIKE '%Franklin Ave%';
""",con)
print(filtered_data)

      id            name  license_id  address_number address_street_name  \
0  16371  Annabel Miller      490173             103        Franklin Ave   

         ssn  
0  318771143  


# 3. View Witness Interviews:
Task: Access the recorded interviews of witnesses conducted after the murder. Gather insights into their statements and potential clues.


In [15]:
filtered_data = pd.read_sql_query("""SELECT *
FROM interview;
""",con)
print(filtered_data)

      person_id                                         transcript
0         28508                ‘I deny it!’ said the March Hare.\n
1         63713                                                 \n
2         86208      way, and the whole party swam to the shore.\n
3         35267  lessons in here? Why, there’s hardly room for ...
4         33856                                                 \n
...         ...                                                ...
4986      37357  Alice did not wish to offend the Dormouse agai...
4987      10206  time,’ she said, ‘than waste it in asking ridd...
4988      14887  I heard a gunshot and then saw a man run out. ...
4989      16371  I saw the murder happen, and I recognized the ...
4990      67318  I was hired by a woman with a lot of money. I ...

[4991 rows x 2 columns]


In [16]:
# Records of first witness
witness_1 = pd.read_sql_query("""SELECT * from interview
WHERE person_id = 14887;
""",con)
print(witness_1)

   person_id                                         transcript
0      14887  I heard a gunshot and then saw a man run out. ...


In [17]:
# Records of second witness
witness_2 = pd.read_sql_query("""SELECT * from interview
WHERE person_id = 16371;
""",con)
print(witness_2)

   person_id                                         transcript
0      16371  I saw the murder happen, and I recognized the ...


# 4. Check Gym Database:
Task: Investigate the gym database using details obtained from the crime scene report and witness interviews. Look for any gym-related information that might be relevant.

There are two tables which contains gym related information.
1. get_fit_now_member
2. get_fit_now_check_in

So, we got 2 clues-

Killer is a man and a member of the gym with a status of gold and having a membership no. starting with 48Z and left in a car with a no. plate of H42W
He was working out in the gym on 9th of Jan


In [18]:
filtered_data = pd.read_sql_query("""SELECT * from get_fit_now_member;
""",con)
print(filtered_data)

        id  person_id                name  membership_start_date  \
0    NL318      65076     Everette Koepke               20170926   
1    AOE21      39426        Noe Locascio               20171005   
2    2PN28      63823  Jeromy Heitschmidt               20180215   
3    0YJ24      80651      Waneta Wellard               20171206   
4    3A08L      32858        Mei Bianchin               20170401   
..     ...        ...                 ...                    ...   
179  2V137      41693      Wendell Dulany               20171219   
180  4KB72      79110          Emile Hege               20170522   
181  48Z7A      28819        Joe Germuska               20160305   
182  48Z55      67318       Jeremy Bowers               20160101   
183  90081      16371      Annabel Miller               20160208   

    membership_status  
0                gold  
1             regular  
2              silver  
3                gold  
4              silver  
..                ...  
179            

In [19]:
filtered_data = pd.read_sql_query("""SELECT * FROM get_fit_now_check_in;
""",con)
print(filtered_data)

     membership_id  check_in_date  check_in_time  check_out_time
0            NL318       20180212            329             365
1            NL318       20170811            469             920
2            NL318       20180429            506             554
3            NL318       20180128            124             759
4            NL318       20171027            418            1019
...            ...            ...            ...             ...
2698         4KB72       20170422           1016            1114
2699         4KB72       20170630            408             885
2700         48Z7A       20180109           1600            1730
2701         48Z55       20180109           1530            1700
2702         90081       20180109           1600            1700

[2703 rows x 4 columns]


In [20]:
# Using the clues where killer having a membership no. starting with 48Z and working out in the gym on 9th of Jan
filtered_data = pd.read_sql_query("""SELECT *
FROM get_fit_now_check_in
WHERE membership_id like "%48Z%" AND check_in_date = 20180109
order by check_in_date;
""",con)
print(filtered_data)

  membership_id  check_in_date  check_in_time  check_out_time
0         48Z7A       20180109           1600            1730
1         48Z55       20180109           1530            1700


# 5. Check Car Details:
Task: Examine the car details associated with the crime scene. Retrieve information about the vehicles present during the incident.




In [21]:
filtered_data = pd.read_sql_query("""SELECT * from drivers_license;
""",con)
print(filtered_data)

           id  age  height eye_color hair_color  gender plate_number  \
0      100280   72      57     brown        red    male       P24L4U   
1      100460   63      72     brown      brown  female       XF02T6   
2      101029   62      74     green      green  female       VKY5KR   
3      101198   43      54     amber      brown  female       Y5NZ08   
4      101255   18      79      blue       grey  female       5162Z1   
...       ...  ...     ...       ...        ...     ...          ...   
10002  999923   19      77     amber      black  female       5L0ZI4   
10003  999940   71      61     green      green    male       1B8QN8   
10004  999981   67      69     brown       blue  female       1684K3   
10005  999986   49      58     green       grey    male       F8F64H   
10006  999993   18      63     black      black  female       6UZO2O   

         car_make    car_model  
0           Acura          MDX  
1        Cadillac          SRX  
2           Scion           xB  
3  

In [22]:
# now, let's check the car details by the above details
filtered_data = pd.read_sql_query("""SELECT *
FROM drivers_license
WHERE plate_number like "%H42W%";
""",con)
print(filtered_data)


       id  age  height eye_color hair_color  gender plate_number   car_make  \
0  183779   21      65      blue     blonde  female       H42W0X     Toyota   
1  423327   30      70     brown      brown    male       0H42W2  Chevrolet   
2  664760   21      71     black      black    male       4H42WR     Nissan   

  car_model  
0     Prius  
1  Spark LS  
2    Altima  


# 6. Personal Details:
Task: Identify and collect personal details mentioned in the previous query. This includes names, addresses, and any additional details.

Since the killer is a man who left in a car with a no. plate of H42W. From the above query there are two male with a plate no. containg H42W. Now we have to check their personal details.





















In [23]:
#checking personal details of both the males from the above query
filtered_data = pd.read_sql_query("""SELECT *
FROM person
WHERE license_id = "423327" OR license_id = "664760";
""",con)
print(filtered_data)

      id            name  license_id  address_number    address_street_name  \
0  51739  Tushar Chandra      664760             312                 Phi St   
1  67318   Jeremy Bowers      423327             530  Washington Pl, Apt 3A   

         ssn  
0  137882671  
1  871539279  


In [24]:
#lets check which of this two are a member of the gym?
filtered_data = pd.read_sql_query("""SELECT *
FROM get_fit_now_member
WHERE person_id = "51739" OR person_id = "67318";
""",con)
print(filtered_data)

      id  person_id           name  membership_start_date membership_status
0  48Z55      67318  Jeremy Bowers               20160101              gold


# To find the mastermind behind this mystery

In [25]:
#There's more to this, reading the transcript of the murderer
filtered_data = pd.read_sql_query("""
SELECT *
FROM interview
WHERE person_id = 67318;
""",con)
print(filtered_data)

   person_id                                         transcript
0      67318  I was hired by a woman with a lot of money. I ...


So, the real mastermind is a woman with a Tesla car and red hair. Using the above clues let find out who's the mastermind behind this murder.

In [26]:
filtered_data = pd.read_sql_query("""
SELECT *
FROM drivers_license
WHERE car_make = "Tesla" AND car_model = "Model S" AND
gender = "female" AND hair_color = "red";
""",con)
print(filtered_data)

       id  age  height eye_color hair_color  gender plate_number car_make  \
0  202298   68      66     green        red  female       500123    Tesla   
1  291182   65      66      blue        red  female       08CM64    Tesla   
2  918773   48      65     black        red  female       917UU3    Tesla   

  car_model  
0   Model S  
1   Model S  
2   Model S  


In [27]:
#personal details of the above three woman are:
filtered_data = pd.read_sql_query("""
SELECT *
FROM person
WHERE license_id = "202298" OR license_id = "291182" OR license_id = "918773";
""",con)
print(filtered_data)

      id              name  license_id  address_number address_street_name  \
0  78881          Red Korb      918773             107         Camerata Dr   
1  90700     Regina George      291182             332           Maple Ave   
2  99716  Miranda Priestly      202298            1883          Golden Ave   

         ssn  
0  961388910  
1  337169072  
2  987756388  


In [28]:
# since the women is rich and attended the SQL Symphony Concert
filtered_data = pd.read_sql_query("""
WITH rich_suspects AS (
    SELECT person.id AS person_id, name, annual_income
    FROM person
    LEFT JOIN income ON person.ssn = income.ssn
    WHERE license_id = "202298" OR license_id = "291182" OR license_id = "918773"
), symphony_attenders AS (
    SELECT person_id, COUNT(1) AS n_checkins
    FROM facebook_event_checkin
    WHERE event_name = 'SQL Symphony Concert'
    AND date LIKE '%201712%'
    GROUP BY person_id
    HAVING n_checkins = 3
)
SELECT name, annual_income
FROM rich_suspects
INNER JOIN symphony_attenders ON rich_suspects.person_id = symphony_attenders.person_id;
""", con)
print(filtered_data)


               name  annual_income
0  Miranda Priestly         310000
